### BERT Model as question answer

In [1]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import os

d:\WorkSpace\Python\Gen-AI-Ground\Google-Bert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasetDir = os.path.join(os.getcwd(), "QnA_Dataset")


In [4]:
pd.read_parquet(os.path.join(datasetDir, "train.parquet")).to_csv(os.path.join(datasetDir, "train.csv"), index=False)

In [86]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

d:\WorkSpace\Python\Gen-AI-Ground\Google-Bert\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Bumblebee\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [87]:
# we must load the model as per our need
model = AutoModelForQuestionAnswering.from_pretrained('distilbert/distilbert-base-uncased')

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
# model understands only digits, this need to convert into digits
# below is for direct tensor object
# inputTokens = tokenizer.encode('how are you feeling today?', return_tensors='pt') # return tensor is pytorch

# model understands only digits, this need to convert into digits
inputTokens = tokenizer('how are you?', 'thi is greeting, so you can say I am good how are you doing ?', return_tensors='pt') # return tensor is pytorch

In [60]:
inputTokens

{'input_ids': tensor([[  101,  2129,  2024,  2017,  1029,   102, 16215,  2072,  2003, 14806,
          1010,  2061,  2017,  2064,  2360,  1045,  2572,  2204,  2129,  2024,
          2017,  2725,  1029,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [61]:
result = model(**inputTokens)

In [62]:
result # now we have to convert the below digits in text

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.2537,  0.2785,  0.4835,  0.5569, -0.3285,  0.7268,  0.1879,  0.0671,
          0.1695, -0.3299,  0.3131,  0.2670,  0.0673,  0.0362, -0.3952,  0.0825,
          0.3303, -0.3242,  0.4738,  0.7419,  0.5587,  0.6743, -0.1732,  0.2737]],
       grad_fn=<CloneBackward0>), end_logits=tensor([[ 0.0436, -0.3200, -0.4852, -0.4312, -0.0375, -0.0793, -0.5397, -0.3525,
         -0.1869, -0.0103, -0.0632,  0.2572, -0.1004,  0.3186,  0.2229, -0.0437,
         -0.3969, -0.2277, -0.5245, -0.5156, -0.4089, -0.2604, -0.0812, -0.2091]],
       grad_fn=<CloneBackward0>), hidden_states=None, attentions=None)

start_logits: denotes the start point of tokenid

and end_logits: end token if as per start token id

In [70]:
result.start_logits[0][0]

tensor(-0.2537, grad_fn=<SelectBackward0>)

In [78]:
answer_start_idx = torch.argmin(result.start_logits)
answer_end_idx = torch.argmin(result.end_logits)

In [73]:
answer_start_idx = result.start_logits[0]
answer_end_idx = result.end_logits[0]

In [80]:
answer_tokens = inputTokens.input_ids[0, answer_start_idx: answer_end_idx +1]


In [81]:
answer = tokenizer.decode(answer_tokens)

In [82]:
answer

''